In [ ]:
import sys
import numpy as np
import scipy.io
import torch
from pyDOE import lhs
from torch import Tensor, ones, stack, load
from torch.autograd import grad
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.pyplot import figure
import pandas as pd
from torch.nn import Module
from torch.utils.data import DataLoader, random_split
from pathlib import Path
import wandb
import time
from tesladatamlppower import TeslaDatasetMlpPower

# Import PINNFramework etc.
# https://github.com/ComputationalRadiationPhysics/NeuralSolvers.git
sys.path.append("NeuralSolvers")  
import PINNFramework as pf

In [ ]:
# Use cuda if it is available, else use the cpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
ds_test = TeslaDatasetMlpPower(device = device, ID = -1, data = "test")
ds_test.df0.shape

## Preprocessing

In [ ]:
# login wandb (optional)
wandb.login()

# Initialize WandB (optional)
wandb.init(name='mlp_power4-reg', 
           project='MLP_project',
           notes='df_x = df0[["battery_temperature","acc_fwd_diff", "battery_level", "outside_temp"]]\
           df_y = df0[["power"]]*normalize', 
           #tags=['...'],
           #entity='...'
           )


# WandB Configurations (optional)
wandb.config.normalize=0.01      
wandb.config.batch_size=2048       
wandb.config.lr = 1e-4

# Create instance of the dataset
ds = TeslaDatasetMlpPower(device = device, data = "train",normalize = wandb.config.normalize)
ds_test = TeslaDatasetMlpPower(device = device, ID = -1, data = "test",normalize = wandb.config.normalize )


# trainloader
trainloader = DataLoader(ds, batch_size=wandb.config.batch_size,shuffle=True)
validloader = DataLoader(ds_test, batch_size=wandb.config.batch_size,shuffle=True)

# MLP model

model = pf.models.MLP(input_size=4,
                      output_size=1, 
                      hidden_size=100, 
                      num_hidden=4, 
                      lb=ds.lb, 
                      ub=ds.ub)

model.to(device)

# optimizer
optimizer = torch.optim.Adam(model.parameters(),lr=wandb.config.lr)
criterion = torch.nn.MSELoss()

# Log the network weight histograms (optional)
wandb.watch(model)

In [ ]:
ds.df0.shape

In [ ]:
def derivative(x, u):

    grads = ones(u.shape, device=u.device) # move to the same device as prediction
    grad_u = grad(u, x, create_graph=True, grad_outputs=grads )[0]
   
    # calculate first order derivatives
    #u_t = grad_u[:, 4]
    #u_t = u_t.reshape(-1, 1)
    
#     u_t = torch.sum(grad_u, 1).reshape(-1, 1)
    
    return grad_u

In [ ]:
# function for saving checkpoints during training
def write_checkpoint(checkpoint_path, epoch, min_mlp_loss, optimizer):
    checkpoint = {}
    checkpoint["epoch"] = epoch
    checkpoint["minimum_mlp_loss"] = min_mlp_loss
    checkpoint["optimizer"] = optimizer.state_dict()
    checkpoint["mlp_model"] = model.state_dict()
    torch.save(checkpoint, checkpoint_path)

## Training

In [ ]:
epochs = 2000
wandb.config.alpha = 1
min_mlp_loss = np.inf
min_valid_loss = np.inf

x_data_plot=[]
y_data_all_plot=[]
y_data_1_plot=[]
y_data_2_plot=[]

# Set fixed random number seed
torch.manual_seed(1234)

begin = time.time()
for epoch in range(epochs):
    # Print epoch
    print(f'Starting epoch {epoch}')

    # Set current and total loss value
    current_loss = 0.0
    total_train_loss = 0.0
    total_train_loss1 = 0.0
    total_train_loss2 = 0.0

    model.train() 
    for i, data in enumerate(trainloader,0):
        #print(i)
        x_batch, y_batch = data     
        optimizer.zero_grad()
        x_batch.requires_grad=True #new
        target = model(x_batch.to(device))
        u_deriv = derivative(x_batch,target) #new
        #print('u_t shape', u_t.shape)
        #print('target shape', target.shape  
        loss1 = criterion(target,y_batch.to(device))
        
#         target2 = torch.zeros_like(u_t)
#         loss2 = criterion(target2.to(device),u_t.to(device))
        loss2 = torch.mean(u_deriv**2) * wandb.config.alpha #new new
        loss = loss1 + loss2 #new
        #print('loss shape', loss)
        
        loss = loss1 + loss2
        
        loss.backward()
        optimizer.step()

        # Print statistics
        current_loss += loss.item()
        total_train_loss += loss.item()
        total_train_loss1 += loss1.item()
        total_train_loss2 += loss2.item()

        if i % 50 == 49:
            print('Loss after mini-batch %5d: %.8f' %
                  (i + 1, current_loss / 50))
            current_loss = 0.0
            
    train_loss = total_train_loss/(i+1)
    loss1 = total_train_loss1/(i+1)
    loss2 = total_train_loss2/(i+1)
    #print("Epoch ", epoch, "Total train Loss ", train_loss)
    
    x_data_plot.append(epoch)
    y_data_all_plot.append(train_loss)
    y_data_1_plot.append(loss1)
    y_data_2_plot.append(loss2)
    
    ######## new
    valid_loss = 0.0
    model.eval()     # Optional when not using Model Specific layer
    for j, data in enumerate(validloader,0):
        x_batch, y_batch = data
        # Forward Pass
        target = model(x_batch.to(device))
        # Find the Loss
        loss = criterion(target,y_batch.to(device))
        # Calculate Loss
        valid_loss += loss.item()

#     print(f'Epoch {epoch} \t Training Loss: {\
#     train_loss / len(trainloader)} \t Validation Loss: {\
#     valid_loss / len(validloader)}')
    
#     print(f'Epoch {epoch} \t Training Loss: {\
#     train_loss} \t Loss 1: {loss1} \t Loss 2: {loss2} \t Validation Loss: {\
#     valid_loss / (j+1)}')
    
    print(f'Epoch {epoch} \t Training Loss: {\
        train_loss:.5f} \t Loss 1: {loss1:.5f} \t Loss 2: {loss2:.5f} \t Validation Loss: {valid_loss / (j+1):.5f}')    

        
    if min_valid_loss > valid_loss:
        print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss:.6f}) \t Saving The Model')
        min_valid_loss = valid_loss
        
        # Saving State Dict
        model_name_path = Path('mlpmodel/best_model_{}_{}.pt'.format(wandb.run.id, wandb.run.name))
        torch.save(model.state_dict(), model_name_path)
        
#         # Saving State Dict
#         torch.save(model.state_dict(), 'saved_model.pth')

    # writing checkpoint
    if (epoch + 1) % 200 == 0:
        checkpoint_path = Path('mlpmodel/checkpoint_{}_{}_{}.pt'.format(wandb.run.id, wandb.run.name, epoch))
        write_checkpoint(checkpoint_path, epoch, min_valid_loss, optimizer)


# ##########old
#     # writing checkpoint
#     if (epoch + 1) % 200 == 0:
#         checkpoint_path = Path('/mlpmodel/checkpoint_{}_{}_{}.pt'.format(wandb.run.id, wandb.run.name, epoch))
#         write_checkpoint(checkpoint_path, epoch, min_mlp_loss, optimizer)
        
#     ########  new  
        
#     # uncomment for saving/tracking the best model and checkpoints
#     # save best model
#     if min_mlp_loss > Loss:
#         print(f'Loss Decreased({min_mlp_loss:.6f}--->{Loss:.6f}) \t Saving The Model')
#         min_mlp_loss = Loss
#         # Saving State Dict
#         model_name_path = Path('/mlpmodel/best_model_{}_{}.pt'.format(wandb.run.id, wandb.run.name))
#         torch.save(model.state_dict(), model_name_path)

#     # writing checkpoint
#     if (epoch + 1) % 200 == 0:
#         checkpoint_path = Path('/mlpmodel/checkpoint_{}_{}_{}.pt'.format(wandb.run.id, wandb.run.name, epoch))
#         write_checkpoint(checkpoint_path, epoch, min_mlp_loss, optimizer)
# #########old

    # Log the loss and accuracy values at the end of each epoch
    wandb.log({
        "Epoch": epoch,
        "Total Loss": train_loss,
        "Loss1 (power)": loss1,
        "Loss2 (regulariser)": loss2,
        "Validation Loss": valid_loss
        })
end = time.time()

print("Training time:", end - begin)

In [ ]:
len(validloader)

In [ ]:
len(trainloader)

In [ ]:
# Make the plot of Total Loss vs epochs
dpi = 360
figure(figsize=(10, 8), dpi = dpi)

plt.plot(x_data_plot,y_data_all_plot)
plt.title('Training loss vs epoch')
plt.xlabel('epoch')
plt.ylabel('Loss')
plt.show()

# Make the plot of the supervised loss
figure(figsize=(10, 8), dpi = dpi)
plt.plot(x_data_plot,y_data_1_plot)
plt.title('Training loss vs epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss1')
plt.show()

# Make the plot of time stability loss
figure(figsize=(10, 8), dpi = dpi)
plt.plot(x_data_plot,y_data_2_plot)
plt.title('Training loss vs epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss2')
plt.show()

## Postprocessing

1) Load the best model


In [ ]:
# Load the best model
PATH = 'mlpmodel/best_model_{}_{}.pt'.format(wandb.run.id, wandb.run.name)
#PATH = 'mlpmodel/best_model_2wwc65h3_mlp_power2-reg.pt' # mlp with regulariser alpha=1
#PATH ='/models/best_model_3a0z7dct_mlp_run.pt'
model.load_state_dict(torch.load(PATH))
#model.load_state_dict(torch.load(PATH, map_location=torch.device('cpu'))) # if there is no gpu available
model.eval()

In [ ]:
# Load checkpoint
#checkpoint = torch.load('mlpmodel/checkpoint_29l1u779_mlp_power_1999.pt')
#model.load_state_dict(checkpoint["mlp_model"])

2. Performance on the training data

In [ ]:
# Make a prediction on trained data
normalize = wandb.config.normalize
pred = model(ds.x.float().to(device)) #GPU
pred = pred.detach().cpu().numpy()/normalize

# ground-truth temperature
gt = ds.y.numpy()/normalize

# Some statistics on the model performance on training dataset
mae = np.sum(np.abs(pred - gt).mean(axis=None))
print('MAE:', mae)

mse = ((gt - pred)**2).mean(axis=None)
print('MSE:', mse)

rel_error = np.linalg.norm(pred - gt) / np.linalg.norm(gt)
print('Relative error (%):', rel_error*100)

# Plot of the prediction vs ground-truth
figure(figsize=(10, 8), dpi= 360)
plt.plot(pred, '--')
plt.plot(gt, '-')
plt.title('Prediction vs ground-truth for the training dataset')
plt.legend(['Prediction', 'ground-truth'])
plt.xlabel('t-idx')
plt.ylabel('Power (kWh)')
plt.show()


## Function for evaluating the model on test data


In [ ]:
# Function for evaluating the model's performance on selected test data
def predict(id):
    print('ID = ', id)
    # Test dataset
    ds_test = TeslaDatasetMlpPower(device = device, ID = id, data = "test")

    # time
    t = ds_test.t

    # prediction
    pred = model(ds_test.x.float().to(device)) #GPU
    pred = pred.detach().cpu().numpy()/normalize

    # ground-truth
    gt = ds_test.y.numpy()

    # Some statistics on the model performance on test dataset
    mae = np.sum(np.abs(pred - gt).mean(axis=None))
    print('MAE:', mae)

    mse = ((gt - pred)**2).mean(axis=None)
    print('MSE:', mse)

    rel_error = np.linalg.norm(pred - gt) / np.linalg.norm(gt)
    print('Relative error (%):', rel_error*100)

    # # 
    # figure(figsize=(10, 8), dpi= 360)
    # plt.plot(pred, '--')
    # plt.plot(gt, '-')
    # plt.title('Prediction vs ground-truth for drive-id = {}'.format(id))
    # plt.legend(['Prediction', 'ground-truth'])
    # plt.xlabel('t-idx')
    # plt.ylabel('Temperature (°C)')
    # plt.show()
    
    # Plot of the prediction vs ground-truth
    figure(figsize=(10, 8), dpi= 360)
    plt.plot(t,pred, '--')
    plt.plot(t,gt, '-')
    plt.title('Prediction vs ground-truth for drive-id = {}'.format(id))
    plt.legend(['Prediction', 'ground-truth'])
    plt.xlabel('t (seconds)')
    plt.ylabel('Power (kW)')
    plt.show()
    return mae,mse,rel_error

3. Performance oof the model on test data

In [ ]:
# Test values = [16,39,47,52,72,81,88]

mae16,mse16,rel_error16 = predict(16)

In [ ]:
mae39,mse39,rel_error39 = predict(39)

In [ ]:
mae47,mse47,rel_error47 = predict(47)

In [ ]:
mae52,mse52,rel_error52 = predict(52)

In [ ]:
mae72,mse72,rel_error72 = predict(72)

In [ ]:
mae81,mse81,rel_error81 = predict(81)

In [ ]:
mae88,mse88,rel_error88 = predict(88)

In [ ]:
# Test values = [16,39,47,52,72,81,88]
mae_avg = (mae16+mae39+mae47+mae52+mae72+mae81+mae88)/7
mse_avg = (mse16+mse39+mse47+mse52+mse72+mse81+mse88)/7
rel_err_avg = (rel_err16+rel_err39+rel_err47+rel_err52+rel_err72+rel_err81+rel_err88)/7

print(mae_avg)
print(mse_avg)
print(rel_err_avg)